In [ ]:
#import libraries

# Date time manipulation libraries
from time import strptime
from time import strftime

# download files
from ftplib import FTP     #conectarme a un ftp para conseguir los datos
import os
# Data-handling and plotting routines libraries
import pygrib    #para trabajar con archivos grib
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib as mpl

# Get handler for figure and setup colormap
def generate_figure():
    # Initiate figure
    # llcrnrlat,llcrnrlon,urcrnrlat,urcrnrlon
    # are the lat/lon values of the lower left and upper right corners
    # of the map.
    # lat_ts is the latitude of true scale.
    # resolution = 'c' means use crude resolution coastlines.
    m = Basemap(projection='merc',llcrnrlat=-60,urcrnrlat=360-85,\
        llcrnrlon=10,urcrnrlon=360-30,lat_ts=20,resolution='c')
    # Draw coastlines, etc.
    m.drawcoastlines()
    m.drawstates()
    m.drawcountries()
    
    # Configure colormap
    cmap = plt.cm.jet
    cmap.set_under((0,0,0,0)) 
    
    return m, cmap
    
# Plot data 
def plot_data(fig_handler, x, y, data, cmap):
    # Latitude and longitudes projection
    x,y = fig_handler(y,x)
    
    # Plot
    cs = fig_handler.contourf(x,y,data,cmap=cmap)
    
    # Configure colorbar
    cb = fig_handler.colorbar(cs,"right", size="5%", pad='2%')
    cb.set_label('°C')
    #cb.set_ticks([0, 10, 20, 30, 40])        


In [ ]:
# Abstract model class
class Model(object):
    # Constructor
    def __init__(self, initial_time, lead_time):
        self.initial_time = initial_time
        self.lead_time = lead_time

# CFS V2 subclass
class CFSV2(Model):
    # Constructor
    def __init__(self,initial_time, lead_time):
        Model.__init__(self,initial_time, lead_time)
    
    # Convert month name to month string number
    def get_datetime_object(self):
                       
        #creates month number as string
        self.initial_time = strftime('%m',strptime(self.initial_time,'%b'))
        

    # Download files 
    def download_files(self):
        URL = 'ftp.cpc.ncep.noaa.gov'
        directorio = '/NMME/clim'
        print ("Connecting FTP ")
        ftp = FTP(URL)     # connect to host, default port
        print (ftp)
        print("Log")
        ftp.login()        # user anonymous, passwd anonymous@ruta = 'ftp.cpc.ncep.noaa.gov/NMME/clim/tmp2m.01.CFSv2.clim.1x1.grb' 
        #manipulo archivos dentro del directorio clim
        ftp.cwd(directorio)   #ingreso al directorio
        nombre_archivo = 'tmpsfc.'+self.initial_time+'.CFSV2.clim.1x1.grb'
        print(nombre_archivo)
        self.get_grib(ftp,nombre_archivo)

    # Downloads grib file
    def get_grib(self,ftp,filename):
        # completar cuando sepa como obtener el grib 
              
        local_filename = os.path.join('data', filename) #ruta donde voy a generar el archivo que se escribe
        file = open(local_filename, 'wb')   #abro el archivo donde se va a alojar
        print("Writing GRIB file")
        ftp.retrbinary('RETR '+ filename, file.write)  #recibo info y guardo en archivo
        file.close()       #cierro archivo
        ftp.quit()      # This is the “polite” way to close a connection  me voy del ftp
        return filename
    
    # Extract data from grib and compute mean
    def extract_data (self, grib_file):
        # Opens file
        #grib_file=Dataset(path_to_file,"r")
        #grib_file = 'tmp2m.12.CFSv2.clim.1x1.grb'
        grbs=pygrib.open(os.path.join('data', grib_file))
        for grb in grbs:
            print (grb)
        grbs.rewind() # rewind the iterator
        t2mens = []
        i_month = sel.lead_time
        grb= grbs[i_month]
        t2mens.append(grb.values)#t2mens = np.array(t2mens)
        t2mens = np.array(t2mens)
        lats, lons = grb.latlons()  # get the lats and lons for the grid.
        lats = lats[:,1]
        lons = lons[1,:]
        # Return values of interest
        return lats, lons, t2mens



"""    
class OTROMODELO(object):
    
    
    
"""